### 거리 계산

In [23]:
import pandas as pd
import geopandas as gpd 

In [24]:
# 버스정류장, 지하철역, 주차장, 카페 등 read [x_nm, x_lat, x_long]
x = pd.read_csv('./parking.csv', encoding='utf-8') 

In [25]:
# 관광지정보 read [name, lon, lat], 530 rows
ts = pd.read_csv('./travel_list.csv', encoding='utf-8') 
ts.drop(columns=['Unnamed: 0'], inplace=True) 

In [26]:
x_gdf = gpd.GeoDataFrame(x, geometry = gpd.points_from_xy(x.p_long,x.p_lat), crs='epsg:4326')  # WGS84(EPSG:4326) :경도위도좌표계 
ts_gdf = gpd.GeoDataFrame(ts, geometry = gpd.points_from_xy(ts.lon, ts.lat), crs='epsg:4326')  
print(x_gdf.head(1))
print(ts_gdf.head(1))

  p_nm      p_lat      p_long                    geometry
0  신선동  35.087052  129.046762  POINT (129.04676 35.08705)
    name        lat         lon                    geometry
0  168계단  35.117469  129.035409  POINT (129.03541 35.11747)


In [27]:
# ts_geo = ts_gdf.to_crs(epsg=5186).iloc[0].geometry # 1번째 관광지 좌표
# .to_crs() (type : geopandas.geodataframe.GeoDataFrame)
# .geometry (type : geopandas.geoseries.GeoSeries) 

# epsg=5186 : TM좌표계로변환 , m단위의 값을 얻기 위해 4186은 경'도'위'도' 도단위임.
# TM좌표계(투영좌표계) : 3차원 위경도를 2차원평면에 나타내기위해 사용. 우리나라 표준
ts_geos = ts_gdf.to_crs(epsg=5186).geometry # 관광지 좌표 리스트
x_geos = x_gdf.to_crs(epsg=5186).geometry # [장소] 좌표 리스트
distances = x_geos.distance(ts_geos[0]) # a관광지와 장소리스트 좌표 간의 거리 리스트(type : Series)
minidx = distances.idxmin() # Series에서 최소값을 가진 인덱스 return
nearest_name = x.iloc[minidx, 0] # 인덱스를 통해 가까운 장소명 반환
nearest_dist = round(distances.iloc[minidx], 1) # 인덱스를 통해 가까운 거리값 반환(1m 단위)

In [28]:
def find_nearest(ts_point):
    distances =  x_geos.distance(ts_point)
    minidx = distances.idxmin()
    nearest_name = x.iloc[minidx][0]
    nearest_dist = round(distances.iloc[minidx], 1)
    return nearest_name, nearest_dist 

In [29]:
ts_gdf['x_nm'] = ts_geos.apply(lambda x: find_nearest(x)[0])
ts_gdf['x_dist'] = ts_geos.apply(lambda x: find_nearest(x)[1])

In [30]:
ts_gdf.to_csv('./find_nearest.csv',encoding='utf-8')

In [31]:
ts_gdf

,name,lat,lon,geometry,x_nm,x_dist
0,168계단,35.117469,129.035409,POINT (129.03541 35.11747),초1-1(주거지),34.5
1,25의용단,35.171540,129.113594,POINT (129.11359 35.17154),망미동노외공영주차장,1528.3
2,40계단 문화관,35.105016,129.034531,POINT (129.03453 35.10502),초량1동업무지,886.4
3,40계단 문화관광테마거리,35.104844,129.037585,POINT (129.03758 35.10484),초량1동업무지,881.6
4,75광장,35.079148,129.053765,POINT (129.05376 35.07915),하늘전망대앞,520.4
...,...,...,...,...,...,...
414,오륙도,35.093793,129.126424,POINT (129.12642 35.09379),용호부두 공영주차장,4355.6
415,이기대 해안산책로,35.132192,129.120646,POINT (129.12065 35.13219),용호부두 공영주차장,415.1
416,장산,35.194415,129.144690,POINT (129.14469 35.19442),반여2동 제2공영,1128.5
417,태종대,35.053070,129.087200,POINT (129.08720 35.05307),태종대유원지부설주차장,1110.2


### 지도로 시각화

In [32]:
import folium
from folium import Marker,Circle

In [33]:
map = folium.Map(location=[35.179665,129.0747635],tiles='openstreetmap',zoom_start=11)

for _, row in ts_gdf.iterrows():
    Circle(location = [row['lat'], row['lon']], #or Marker
           radius = 200,
           popup=row['name'],
        #    icon=folium.Icon(color='blue',icon='cloud')
          ).add_to(map)
map